In [53]:
import pandas as pd
import numpy as np
from ordered_set import OrderedSet


from util.util import get_root_dir

root_dir = get_root_dir()
proj_dir = root_dir + "/surrogate/plastic_train9/"
df_samples1 = pd.read_csv(proj_dir + "lhs_samples.csv")
df_samples1.index += 1  # Shifts the index to start from 1 instead of 0
df_samples = pd.concat([df_samples1], axis=0)
len(df_samples), len(df_samples.columns)

(1000, 3)

In [54]:
df_samples

,Light,Heavy,Total
1,0.742473,0.257527,80518.531719
2,0.172589,0.827411,119382.800381
3,0.413258,0.586742,91002.818480
4,0.612487,0.387513,98887.560782
5,0.679732,0.320268,103028.958086
...,...,...,...
996,0.475841,0.524159,118817.668714
997,0.442682,0.557318,68625.703466
998,0.427833,0.572167,75712.965346
999,0.320141,0.679859,100671.879140


In [55]:
columns_list = ["RR", "STAGES", "COND", "REB"]

df_outputs1 = pd.read_csv(proj_dir + "result3.csv", header=None)
df_outputs1.columns = columns_list
df_outputs1.index += 1  # Shifts the index to start from 1 instead of 0

df_outputs = pd.concat([df_outputs1], axis=0)
# df_outputs.columns = df_outputs.iloc[0]
# df_outputs = df_outputs.drop(df_outputs.index[0])
len(df_outputs), len(df_outputs.columns)
df_outputs

,RR,STAGES,COND,REB
1,1.537637,25.0,11.327516,13.189384
2,4.583033,25.0,8.965640,12.326913
3,2.379468,25.0,9.590911,11.959476
4,1.775840,25.0,12.589590,14.989488
5,1.643030,25.0,13.838106,16.277434
...,...,...,...,...
996,2.142537,25.0,13.366216,16.393061
997,2.260770,25.0,7.463675,9.231989
998,2.318878,25.0,8.106291,10.067155
999,2.878570,25.0,9.497785,12.200846


In [56]:
output_data = df_outputs.drop(['STAGES'], axis=1).to_numpy()
output_data

array([[ 1.53763683, 11.3275157 , 13.1893837 ],
       [ 4.58303323,  8.9656399 , 12.3269134 ],
       [ 2.37946762,  9.59091115, 11.9594763 ],
       ...,
       [ 2.31887835,  8.10629148, 10.0671552 ],
       [ 2.87856959,  9.49778452, 12.2008463 ],
       [ 2.28096385,  7.16314121,  8.87271548]])

In [57]:
input_data_raw = df_samples.to_numpy().T

input_data = np.array([input_data_raw[0] * input_data_raw[2], input_data_raw[1] * input_data_raw[2]]).T
input_data

array([[59782.835801  , 20735.695918  ],
       [20604.15813496, 98778.64224604],
       [37607.64275941, 53395.17572059],
       ...,
       [32392.50510288, 43320.46024312],
       [32229.19605976, 68442.68308024],
       [28968.96082081, 37257.76542419]])

In [58]:
np.shape(input_data), np.shape(output_data)

((1000, 2), (1000, 3))

In [59]:
type(input_data[0][0]), type(output_data[0][0])

(numpy.float64, numpy.float64)

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# R^2: 0.9546, Seed: 1703573223 ; Split:0.6; Archi: 4->20, 20_>20, 20->12, 12->3, epoch: 2.5e4, lr=1e-4

random_state = np.random.RandomState()  # Create a random state object
random_seed = random_state.randint(0, 2**31 - 1 )  # Generate a random seed
# random_seed = 1703573223
print(random_seed)

# Splitting the data into training and testing datasets
split_ratio = 0.7
X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, train_size=split_ratio, random_state=random_seed)

# Setting the device
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# device = torch.device('cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

# Neural Network Definition
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2,15)
        self.fc2 = nn.Linear(15,12)
        self.fc3 = nn.Linear(12,10)
        self.fc4 = nn.Linear(10,3)  # Outputting 2 since there are 2 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

1692101104


In [61]:
from sklearn.model_selection import KFold

model = SimpleNN().to(device)
criterion = nn.MSELoss()
learning_rate = 5e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

loss_data = []

# Training the model
num_epochs = int(4e4)
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_data.append(loss.item())
    if epoch % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/40000], Loss: 1269137.7500
Epoch [1001/40000], Loss: 45.4585
Epoch [2001/40000], Loss: 15.1307
Epoch [3001/40000], Loss: 7.9352
Epoch [4001/40000], Loss: 4.3802
Epoch [5001/40000], Loss: 3.6102
Epoch [6001/40000], Loss: 3.2312
Epoch [7001/40000], Loss: 3.0245
Epoch [8001/40000], Loss: 2.8305
Epoch [9001/40000], Loss: 8.0312
Epoch [10001/40000], Loss: 2.5772
Epoch [11001/40000], Loss: 2.4505
Epoch [12001/40000], Loss: 2.3150
Epoch [13001/40000], Loss: 3.4559
Epoch [14001/40000], Loss: 1.9662
Epoch [15001/40000], Loss: 1.7816
Epoch [16001/40000], Loss: 1.5911
Epoch [17001/40000], Loss: 1.4581
Epoch [18001/40000], Loss: 1.3579
Epoch [19001/40000], Loss: 2.3968
Epoch [20001/40000], Loss: 1.2109
Epoch [21001/40000], Loss: 1.1442
Epoch [22001/40000], Loss: 1.0599
Epoch [23001/40000], Loss: 1.2099
Epoch [24001/40000], Loss: 0.9507
Epoch [25001/40000], Loss: 0.9068
Epoch [26001/40000], Loss: 0.8624
Epoch [27001/40000], Loss: 0.8302
Epoch [28001/40000], Loss: 0.9509
Epoch [29001/40000]

In [62]:
print(random_seed)

1692101104


In [63]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluating the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    y_true = y_test_tensor.cpu().numpy()
    y_pred = test_outputs.cpu().numpy()

    # Calculate MSE and RMSE
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)  # Calculate the Root Mean Squared Error

    # Calculate the Variance (for NMSE)
    variance = np.var(y_true)

    # Calculate NMSE
    nmse = mse / variance if variance != 0 else 'undefined'

    # Calculate R-squared
    r_squared = r2_score(y_true, y_pred)

    # Calculate NRMSE
    range_of_data = y_true.max() - y_true.min()
    nrmse = rmse / range_of_data if range_of_data != 0 else 'undefined'

    # Print out the metrics
    print(f'MSE: {mse:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'NMSE: {nmse}')
    print(f'R-squared: {r_squared:.4f}')
    print(f'NRMSE: {nrmse}')


MSE: 0.7952
RMSE: 0.8918
NMSE: 0.02563469670712948
R-squared: 0.9563
NRMSE: 0.01808083988726139


In [64]:
# Assuming the R-squared value and other details are calculated/stored in variables
r_squared = r_squared
architecture = "2->15, 15->10, 10->12, 12->3"

# Open a text file to write the parameters
with open('ANN_parameters_frac3.txt', 'w') as file:
    # Write the additional information
    file.write(f" MSE: {mse:.4f}, RMSE: {rmse:.4f}, R^2: {r_squared:.4f}, Seed: {random_seed}; Split: {split_ratio}; Archi: {architecture}; Epoch: {num_epochs}; LR: {learning_rate}\n\n")

    # Write the model parameters
    for name, param in model.named_parameters():
        file.write(f"{name}:\n")
        param_values = param.data.cpu().numpy()
        file.write(f"{param_values}\n\n")

print("Model parameters and details saved to ANN_parameters.txt")


Model parameters and details saved to ANN_parameters.txt


In [65]:
# Save X_test to a CSV file
np.savetxt("X_test_frac3.csv", X_test, delimiter=",")
np.savetxt("X_train_frac3.csv", X_test, delimiter=",")

# Save y_test to a CSV file
np.savetxt("y_test_frac3.csv", y_test, delimiter=",")
np.savetxt("y_train_frac3.csv", X_test, delimiter=",")

print("X_test and y_test have been saved to X_test.csv and y_test.csv respectively.")


X_test and y_test have been saved to X_test.csv and y_test.csv respectively.
